
# Exercise 1.1.2
## Classification of MNIST digits with a fully-connected neural network

In this exercise we will classify [MNIST digits](https://en.wikipedia.org/wiki/MNIST_database) using a fully-connected neural network

We start by importing the modules that we need

In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

We check that we have a GPU available

In [2]:
if torch.cuda.is_available():
    print("The code will run on GPU. This is important so things run faster.")
else:
    print("The code will run on CPU. You should probably not do this.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

The code will run on GPU. This is important so things run faster.


The MNIST dataset is a built-in dataset in PyTorch (it is a very common dataset to test algorithms on). We import it, and set our minibatch size, by

In [3]:
batch_size = 64
trainset = datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
testset = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

You shouild implement a fully-connected network to classify the digits. It should contain 1 hidden layer with 20 units. You should use ReLU activation function on the hidden layer. 

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.fc1 = nn.Sequential(
            nn.Linear(28*28,500),
            nn.ReLU()
        )#... # you should fill in this line
        
        self.fc2 = nn.Sequential(
            nn.Linear(500,10),
            nn.Softmax(dim = 1)
            )
        
    def forward(self, x):
        x = x.view(x.size(0),-1)
        #reshaping x so it becomes flat, except for the first dimension (which is the minibatch)
        x = self.fc1(x)
        #x = F.ReLU(x)
        x = self.fc2(x)
        return x

We instantiate a copy of our network and transfer it to the GPU if it's available

In [5]:
model = Network()
model.to(device)
#Initialize the optimer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [6]:
#We train the network for five epochs

In [7]:
num_epochs = 10

for epoch in tqdm(range(num_epochs), unit='epoch'):
    #For each epoch
    train_correct = 0
    for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        #Zero the gradients computed for each weight
        optimizer.zero_grad()
        #Forward pass your image through the network
        output = model(data)
        #Compute the loss
        loss = F.nll_loss(torch.log(output), target)
        #Backward pass through the network
        loss.backward()
        #Update the weights
        optimizer.step()
        
        #Compute how many were correctly classified
        predicted = output.argmax(1)
        train_correct += (target==predicted).sum().cpu().item()
    #Comput the test accuracy
    test_correct = 0
    for data, target in test_loader:
        data = data.to(device)
        with torch.no_grad():
            output = model(data)
        predicted = output.argmax(1).cpu()
        test_correct += (target==predicted).sum().item()
    train_acc = train_correct/len(trainset)
    test_acc = test_correct/len(testset)
    print("Accuracy train: {train:.1f}%\t test: {test:.1f}%".format(test=100*test_acc, train=100*train_acc))

Accuracy train: 88.5%	 test: 92.8%


Accuracy train: 93.9%	 test: 94.6%


Accuracy train: 95.6%	 test: 96.0%


Accuracy train: 96.5%	 test: 96.6%


Accuracy train: 97.1%	 test: 97.2%


Accuracy train: 97.6%	 test: 97.1%


Accuracy train: 97.9%	 test: 97.3%


Accuracy train: 98.2%	 test: 97.6%


Accuracy train: 98.4%	 test: 97.7%


Accuracy train: 98.6%	 test: 97.7%



You should now have a model that has about 95 % accuracy on the test set.
Try to get an even better accuracy. You can
* Change the number of hidden layers
* Change the number of units in the hidden layers
* Try using sigmoid instead of relu activation
* Try changing the learning rate

How large accuracy can you get?